# **RECIPE REVIEW ANNOTATON WITH PYABSA**

This notebook presents 
- Upload of Reviews file from Recipe and Reviews dataset on Kaggle
- Extraction of reviews column from the Reviews dataset
- Data Cleaning of Reviews Column
    - NAN and empty rows are dropped
    - contractions are expanded
    - regular expressions are removed
- Load the cleaned review list into pyabsa
- Get .json file output of annotated reviews

PyABSA will not run on colab if you are using the free GPU resource as it is not sufficient to run the annotation process. In this study, we used the free GPU resource provided by Kaggle and it worked. If you run this process your local machine, it is important to be sure that your GPU resource is sufficient to handle this process. 

In [1]:
#import pyabsa for annotation
pip install -U pyabsa

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#import easpect term extraction library from pyabsa, we use the aspect term extraction library
from pyabsa import AspectTermExtraction as ATEPC, available_checkpoints

In [2]:
#import other libraries for dataframe manipulation and data cleaning
import re
import pandas as pd
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
import spacy

import numpy as np
from pandas.io.json import json_normalize


<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/s

In [3]:
#read and upload file from kaggle
df=pd.read_csv('/kaggle/input/foodcom-recipes-and-reviews/reviews.csv', error_bad_lines=False, engine="python")
df.head()

/tmp/ipykernel_390/644516016.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df=pd.read_csv('/kaggle/input/foodcom-recipes-and-reviews/reviews.csv', error_bad_lines=False, engine="python")


,ReviewId,RecipeId,AuthorId,AuthorName,Rating,Review,DateSubmitted,DateModified
0,2,992,2008,gayg msft,5,better than any you can get at a restaurant!,2000-01-25T21:44:00Z,2000-01-25T21:44:00Z
1,7,4384,1634,Bill Hilbrich,4,"I cut back on the mayo, and made up the differ...",2001-10-17T16:49:59Z,2001-10-17T16:49:59Z
2,9,4523,2046,Gay Gilmore ckpt,2,i think i did something wrong because i could ...,2000-02-25T09:00:00Z,2000-02-25T09:00:00Z
3,13,7435,1773,Malarkey Test,5,easily the best i have ever had. juicy flavor...,2000-03-13T21:15:00Z,2000-03-13T21:15:00Z
4,14,44,2085,Tony Small,5,An excellent dish.,2000-03-28T12:51:00Z,2000-03-28T12:51:00Z


Data Cleaning

In [ ]:
import sys
!{sys.executable} -m pip install contractions
import contractions

In [ ]:
#Removing Contractions
df['Review'].apply(lambda x: " ".join(contractions.fix(x) for x in str(x).split()))

In [4]:
reviews = df['Review']

In [5]:
#transform dataframe to list, the datatype for pyabsa input is a list
reviews = reviews.tolist()

In [9]:
#removing regular expressions
regex = re.compile(r'[\n\r\t]')
new_revs = []

for word in reviews:
    #print(word)
    #re.findall(r',?([^,]+)(?:,|\r\n)', word)
    sent = regex.sub(" ", str(word))
    str(word).replace('\\','')
    new_revs.append(sent)
    

In [ ]:
from pyabsa import make_ABSA_dataset 
# refer to the comments in this function for detailed usage
make_ABSA_dataset(str(new_revs), checkpoint='english')

[2023-08-14 18:05:35] (2.3.3) No files found! Please make sure your dataset names end with ".ignore"


In [ ]:
new_revs

In [ ]:
# you can view all available checkpoints by calling available_checkpoints()
checkpoint_map = available_checkpoints()

aspect_extractor = ATEPC.AspectExtractor('multilingual',
                                         auto_device=True,  # False means load model on CPU
                                         cal_perplexity=True,
                                         )

# instance inference
#aspect_extractor.predict(["An excellent dish.", "Very easy to make and great to freeze for popsiciles"],
                         #save_result=True,
                         #print_result=True,  # print the result
                         #ignore_error=True,  # ignore the error when the model cannot predict the input
                         #)

inference_source = new_revs
atepc_result = aspect_extractor.batch_predict(target_file=inference_source,  #
                                             save_result=True,
                                             print_result=True,  # print the result
                                             pred_sentiment=True,  # Predict the sentiment of extracted aspect terms
                                             )

#print(atepc_result)

In [ ]:
%%bash
# we can use %%bash magic to print a preview of our file

head ../kaggle/working/Aspect Term Extraction and Polarity Classification.FAST_LCF_ATEPC.result.json